In [ ]:
!pip install skorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.3/239.3 kB 3.6 MB/s eta 0:00:00


In [ ]:
import datetime
import itertools
import numpy as np
import os
import pandas as pd
import patsy
import time

import sklearn
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch import optim
from torch.autograd import Variable

from skorch import NeuralNet
from skorch.helper import predefined_split, SliceDataset
from skorch.callbacks import BatchScoring, Checkpoint, EarlyStopping, EpochScoring, LRScheduler, TensorBoard, ProgressBar

import tensorflow as tf
from tensorflow import summary


In [ ]:
identifier = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(identifier)
print(device)

cuda


In [ ]:
if not os.path.exists('ml-100k'):
    !wget http://files.grouplens.org/datasets/movielens/ml-100k.zip
    !unzip -o ml-100k.zip

--2024-01-11 02:00:56--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  9.92MB/s    in 0.5s    

2024-01-11 02:00:57 (9.92 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

### Dataset

In [ ]:
genre_cols = [
    "genre_unknown", "Action", "Adventure", "Animation", "Children", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "FilmNoir", "Horror",
    "Musical", "Mystery", "Romance", "SciFi", "Thriller", "War", "Western"
]

In [ ]:
class rsdataset(Dataset):
    def __init__(self, usersfile, moviesfile, ratingsfile, nrows=None):

        # Read files
        self.movies = pd.read_csv(moviesfile, sep='|', names=['MovieID', 'Title', 'date', 'video_rl_date', 'link']+genre_cols, engine='python', encoding='latin-1')
        self.users = pd.read_csv(usersfile, sep='|', names=['UserID', 'Age', 'Gender', 'Occupation', 'Zipcode'], engine='python', encoding='latin-1')
        self.ratings = pd.read_csv(ratingsfile, sep='\t', names=['UserID', 'MovieID', 'Rating', 'Timestamp'], engine='python', nrows=nrows, encoding='latin-1')

        df2 = self.movies[genre_cols]
        df2['Genre'] = df2.apply(lambda row: ', '.join(row.index[row == 1]), axis=1)
        self.movies['Genre'] = df2['Genre']
        self.movies = self.movies.drop(genre_cols, axis = 1)
        # self.movies['Genre'] = self.movies['Genre'].map(genre_dict)
        bins = [0, 18, 25, 35, 45, 50, 56, 100]
        labels = [1, 18, 25, 35, 45, 50, 56]
        self.users['Age'] = pd.cut(self.users['Age'], bins=bins, labels=labels, right=False)
        assert self.users['UserID'].nunique() >= self.ratings['UserID'].nunique(), 'UserID with unknown information'
        assert self.movies['MovieID'].nunique() >= self.ratings['MovieID'].nunique(), 'Movies with unknown information'

        self.users_emb_columns = []
        self.users_ohe_columns = []
        self.movies_emb_columns = []
        self.movies_ohe_columns = []
        self.interact_columns = []

        self.movies = self.movies.drop(['date', 'video_rl_date', 'link'], axis= 1)
        self.nusers = self.ratings['UserID'].nunique()
        self.nmovies = self.ratings['MovieID'].nunique()

        self.y_range = (self.ratings['Rating'].min(), self.ratings['Rating'].max())

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):

        return (((self.users_emb[idx])),
                ((self.users_ohe[idx])),
                ((self.movies_emb[idx])),
                ((self.movies_ohe[idx])),
                ((self.interact[idx]))), (self.y[idx])

    def to_tensor(self):
        self.users_emb = torch.from_numpy(self.ratings[self.users_emb_columns].values)
        self.users_ohe = torch.tensor(self.ratings[self.users_ohe_columns].values, dtype=torch.float)
        self.movies_emb = torch.from_numpy(self.ratings[self.movies_emb_columns].values)
        self.movies_ohe = torch.tensor(self.ratings[self.movies_ohe_columns].values, dtype=torch.float)
        self.interact = torch.from_numpy(self.ratings[self.interact_columns].values)
        self.y = torch.tensor(self.y.values, dtype=torch.float)

In [ ]:
train = rsdataset('ml-100k/u.user', 'ml-100k/u.item', 'ml-100k/u.data', nrows=None)

<ipython-input-6-f5a438cfccf0>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Genre'] = df2.apply(lambda row: ', '.join(row.index[row == 1]), axis=1)


In [ ]:
train.users

,UserID,Age,Gender,Occupation,Zipcode
0,1,18,M,technician,85711
1,2,50,F,other,94043
2,3,18,M,writer,32067
3,4,18,M,technician,43537
4,5,25,F,other,15213
...,...,...,...,...,...
938,939,25,F,student,33319
939,940,25,M,administrator,02215
940,941,18,M,student,97229
941,942,45,F,librarian,78209


In [ ]:
train.movies

,MovieID,Title,Genre
0,1,Toy Story (1995),"Animation, Children, Comedy"
1,2,GoldenEye (1995),"Action, Adventure, Thriller"
2,3,Four Rooms (1995),Thriller
3,4,Get Shorty (1995),"Action, Comedy, Drama"
4,5,Copycat (1995),"Crime, Drama, Thriller"
...,...,...,...
1677,1678,Mat' i syn (1997),Drama
1678,1679,B. Monkey (1998),"Romance, Thriller"
1679,1680,Sliding Doors (1998),"Drama, Romance"
1680,1681,You So Crazy (1994),Comedy


In [ ]:
train.ratings

,UserID,MovieID,Rating,Timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


### Preprocessing of dataset

In [ ]:
train.ratings = train.ratings.merge(train.movies, left_on='MovieID', right_on='MovieID')
train.movies = train.ratings[train.movies.columns]

train.ratings = train.ratings.merge(train.users, left_on='UserID', right_on='UserID')
train.users = train.ratings[train.users.columns]

train.y = train.ratings['Rating']

In [ ]:
# Label Encode users
columns = ['UserID', 'Gender', 'Age', 'Occupation']
train.ratings[columns] = train.ratings[columns].apply(preprocessing.LabelEncoder().fit_transform)
train.users_emb_columns = train.users_emb_columns + columns

In [ ]:
# Label Encode movies
columns = ['MovieID']
train.ratings[columns] = train.ratings[columns].apply(preprocessing.LabelEncoder().fit_transform)
train.movies_emb_columns = train.movies_emb_columns + columns

In [ ]:
# One Hot Encode users
columns = ['Gender', 'Age', 'Occupation']
ohe = preprocessing.OneHotEncoder(categories='auto', sparse=False, dtype='uint8')
ohe.fit(train.ratings[columns])
train.ratings = pd.concat([train.ratings, pd.DataFrame(data=ohe.transform(train.ratings[columns]), columns=ohe.get_feature_names_out(columns))], axis=1)
train.users_ohe_columns = ohe.get_feature_names_out(columns)

assert train.ratings[train.users_ohe_columns].max().max()<=1, 'Error with ohe columns'

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
# One Hot Encode movies
genres = ["genre_unknown", "Action", "Adventure", "Animation", "Children", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "FilmNoir", "Horror",
    "Musical", "Mystery", "Romance", "SciFi", "Thriller", "War", "Western"]

for genre in genres:
    genre = genre.replace('-', '')
    column = str(genre)
    train.ratings[column] = train.ratings['Genre'].apply(lambda x: 1 if genre in x else 0)
    train.movies_ohe_columns.append(column)

assert train.ratings[train.movies_ohe_columns].max().max()<=1, 'Error with ohe columns'

In [ ]:
int_genres_gender = ""
for genre in train.movies_ohe_columns:
    int_genres_gender = int_genres_gender + '+' +genre + ':Gender'

int_genres_age = ""
for genre in train.movies_ohe_columns:
    int_genres_age = int_genres_age + '+' + genre + ':Age'

interact = patsy.dmatrix("0 + Gender:Age + Gender:Occupation + Age:Occupation"+int_genres_gender+int_genres_age, data=train.ratings.astype('object'), return_type='dataframe').astype('int8')
interact = interact.astype('uint8')
train.ratings = pd.concat([train.ratings, interact], axis=1)
train.interact_columns = interact.columns

In [ ]:
interact.columns[interact.iloc[0] == 1]


Index(['Gender[1]:Age[4]', 'Gender[1]:Occupation[T.20]',
       'Age[T.4]:Occupation[T.20]', 'Comedy[T.1]:Gender[1]',
       'Comedy[T.1]:Age[T.4]'],
      dtype='object')

In [ ]:
# Drop unused columns
train.movies.drop(['Title', 'Genre'], inplace=True, axis=1)
train.ratings.drop(['Title', 'Genre', 'Zipcode'], inplace=True, axis=1)

In [ ]:
train.to_tensor()

### DataLoaders

In [ ]:
# Split
train_size = int(0.8 * len(train))
test_size = len(train) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(train, [train_size, test_size])

# Create dataloaders
dataloaders = {}
dataloaders['train'] = torch.utils.data.DataLoader(train_dataset, batch_size=4096, shuffle=True)
dataloaders['valid'] = torch.utils.data.DataLoader(valid_dataset, batch_size=4096, shuffle=True)

### Define Pytorch models

In [ ]:
class deepnwide(nn.Module):

    def __init__(self, users_emb, movies_emb, users_ohe, movies_ohe, interact, size_emb, y_range, dropout, linear_size= 500):
        super().__init__()

        self.name = 'deepnwide'
        self.y_range = y_range

        # wide part

        # deep
        self.emb_UserID = nn.Embedding(len(torch.unique(users_emb[:, 0])), size_emb)
        self.emb_UserID.weight.data.uniform_(-.01, .01)
        self.emb_Gender = nn.Embedding(len(torch.unique(users_emb[:, 1])), size_emb)
        self.emb_Gender.weight.data.uniform_(-.01, .01)
        self.emb_Age = nn.Embedding(len(torch.unique(users_emb[:, 2])), size_emb)
        self.emb_Age.weight.data.uniform_(-.01, .01)
        self.emb_Occupation = nn.Embedding(len(torch.unique(users_emb[:, 3])), size_emb)
        self.emb_Occupation.weight.data.uniform_(-.01, .01)
        self.emb_MovieID = nn.Embedding(len(torch.unique(movies_emb[:, 0])), size_emb)
        self.emb_MovieID.weight.data.uniform_(-.01, .01)

        # hidden layers
        self.h1 = nn.Linear(5 * size_emb, linear_size)
        self.h2 = nn.Linear(linear_size, linear_size)
        self.h3 = nn.Linear(linear_size, linear_size)

        # Dropout layers
        self.dropout1 = nn.Dropout(p=dropout)
        self.dropout2 = nn.Dropout(p=dropout)
        self.dropout3 = nn.Dropout(p=dropout)

        # final dense layer
        self.last_layer = nn.Linear((interact.shape[1]) + (movies_ohe.shape[1]) + (linear_size), 1)


    def forward(self, X):
        # Assign data
        user_emb = X[0]
        user_ohe = X[1]
        movie_emb = X[2]
        movie_ohe = X[3]
        interact = X[4]

        UserID = user_emb[:, 0]
        Gender = user_emb[:, 1]
        Age = user_emb[:, 2]
        Occupation = user_emb[:, 3]
        MovieID = movie_emb[:, 0]

        UserID = self.emb_UserID(UserID)
        Gender = self.emb_Gender(Gender)
        Age = self.emb_Age(Age)
        Occupation = self.emb_Occupation(Occupation)
        MovieID = self.emb_MovieID(MovieID)

        emb = torch.cat([UserID,
                         Age,
                         Gender,
                         Occupation,
                         MovieID],
                         dim=1)

        emb = F.relu(self.dropout1(self.h1(emb)))
        emb = F.relu(self.dropout2(self.h2(emb)))
        emb = F.relu(self.dropout3(self.h3(emb)))

        result = self.last_layer(torch.cat([interact.float(), movie_ohe.float(), emb.float()], dim=1))

        return (torch.sigmoid(result) * (self.y_range[1]-self.y_range[0]) + self.y_range[0]).squeeze()


model = deepnwide(train.users_emb, train.movies_emb, train.users_ohe, train.movies_ohe, train.interact, 60, train.y_range, 0.5, 100)
model.to(device)
print(model)

deepnwide(
  (emb_UserID): Embedding(943, 60)
  (emb_Gender): Embedding(2, 60)
  (emb_Age): Embedding(7, 60)
  (emb_Occupation): Embedding(21, 60)
  (emb_MovieID): Embedding(1682, 60)
  (h1): Linear(in_features=300, out_features=100, bias=True)
  (h2): Linear(in_features=100, out_features=100, bias=True)
  (h3): Linear(in_features=100, out_features=100, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (dropout3): Dropout(p=0.5, inplace=False)
  (last_layer): Linear(in_features=445, out_features=1, bias=True)
)


In [ ]:
class twoembeds(torch.nn.Module):

    def __init__(self, size_emb, y_range):
        super().__init__()

        # set name of model
        self.name = 'twoembeds'
        self.y_range = y_range

        # User and movie embeddings
        self.emb_UserID = nn.Embedding(train.nusers, size_emb)
        self.emb_MovieID = nn.Embedding(train.nmovies, size_emb)
        self.emb_UserID.weight.data.uniform_(-.01, .01)
        self.emb_MovieID.weight.data.uniform_(-.01, .01)

        # User and movie embeddings bía
        self.emb_UserID_b = nn.Embedding(train.nusers, 1)
        self.emb_MovieID_b = nn.Embedding(train.nmovies, 1)
        self.emb_UserID_b.weight.data.uniform_(-.01, .01)
        self.emb_MovieID_b.weight.data.uniform_(-.01, .01)


    def forward(self, X):
        user_emb = X[0]
        user_ohe = X[1]
        movie_emb = X[2]
        movie_ohe = X[3]

        UserID = user_emb[:, 0]
        MovieID = movie_emb[:, 0]

        user_emb = self.emb_UserID(UserID)
        movie_emb = self.emb_MovieID(MovieID)

        mult = (user_emb * movie_emb).sum(1)

        # add bias
        multb = mult + self.emb_UserID_b(UserID).squeeze() + self.emb_MovieID_b(MovieID).squeeze()

        multb = multb.float()

        return (torch.sigmoid(multb) * (self.y_range[1]-self.y_range[0]) + self.y_range[0]).squeeze()

        return multb


model = twoembeds(15, train.y_range)
model.to(device)
print(model)

twoembeds(
  (emb_UserID): Embedding(943, 15)
  (emb_MovieID): Embedding(1682, 15)
  (emb_UserID_b): Embedding(943, 1)
  (emb_MovieID_b): Embedding(1682, 1)
)


In [ ]:
class ncf(torch.nn.Module):

    def __init__(self, users_emb, movies_emb, users_ohe, movies_ohe, interact, size_emb, dropout, linear_size, y_range):
        super().__init__()

        # set name of model
        self.name = 'ncf'
        self.y_range = y_range

        ### GMF part
        # user embeddings
        self.gmf_embuserid = nn.Embedding(len(torch.unique(users_emb[:, 0])), size_emb)
        self.gmf_embuserid.weight.data.uniform_(-.01, .01)
        self.gmf_embgender = nn.Embedding(len(torch.unique(users_emb[:, 1])), size_emb)
        self.gmf_embgender.weight.data.uniform_(-.01, .01)
        self.gmf_embage = nn.Embedding(len(torch.unique(users_emb[:, 2])), size_emb)
        self.gmf_embage.weight.data.uniform_(-.01, .01)
        self.gmf_embocc = nn.Embedding(len(torch.unique(users_emb[:, 3])), size_emb)
        self.gmf_embocc.weight.data.uniform_(-.01, .01)
        # movie embeddings
        self.gmf_embmovieid = nn.Embedding(len(torch.unique(movies_emb[:, 0])), size_emb*4-len(train.movies_ohe_columns))
        self.gmf_embmovieid.weight.data.uniform_(-.01, .01)


        ### MLP part
        # user embeddings
        self.mlp_embuserid = nn.Embedding(len(torch.unique(users_emb[:, 0])), size_emb)
        self.mlp_embuserid.weight.data.uniform_(-.01, .01)
        self.mlp_embgender = nn.Embedding(len(torch.unique(users_emb[:, 1])), size_emb)
        self.mlp_embgender.weight.data.uniform_(-.01, .01)
        self.mlp_embage = nn.Embedding(len(torch.unique(users_emb[:, 2])), size_emb)
        self.mlp_embage.weight.data.uniform_(-.01, .01)
        self.mlp_embocc = nn.Embedding(len(torch.unique(users_emb[:, 3])), size_emb)
        self.mlp_embocc.weight.data.uniform_(-.01, .01)
        # movie embeddings
        self.mlp_embmovieid = nn.Embedding(len(torch.unique(movies_emb[:, 0])), size_emb)
        self.mlp_embmovieid.weight.data.uniform_(-.01, .01)
        # hidden layers
        self.h1 = nn.Linear(5*size_emb+len(train.movies_ohe_columns), linear_size)
        self.h2 = nn.Linear(linear_size, int(linear_size/2))
        #self.h3 = nn.Linear(linear_size, linear_size)
        # Dropout layers
        self.dropout1 = nn.Dropout(p=dropout)
        self.dropout2 = nn.Dropout(p=dropout)
        #self.dropout3 = nn.Dropout(p=dropout)

        # final dense layer
        self.last_layer = nn.Linear(size_emb*4+int(linear_size/2), 1)

    def forward(self, X):
        user_emb = X[0]
        user_ohe = X[1]
        movie_emb = X[2]
        movie_ohe = X[3]

        UserID = user_emb[:, 0]
        Gender = user_emb[:, 1]
        Age = user_emb[:, 2]
        Occupation = user_emb[:, 3]
        MovieID = movie_emb[:, 0]

        # GMF part
        gmf_embuserid = self.gmf_embuserid(UserID)
        gmf_embgender = self.gmf_embgender(Gender)
        gmf_embage = self.gmf_embage(Age)
        gmf_embocc = self.gmf_embocc(Occupation)
        gmf_embmovieid = self.gmf_embmovieid(MovieID)

        gmf_user_vector = torch.cat([gmf_embuserid,
                                    gmf_embgender,
                                    gmf_embage,
                                    gmf_embocc],
                                    dim=1)

        gmf_movie_vector = torch.cat([gmf_embmovieid, movie_ohe], 1)

        gmf_vector = (gmf_user_vector * gmf_movie_vector)


        # MLP part
        mlp_embuserid = self.mlp_embuserid(UserID)
        mlp_embgender = self.mlp_embgender(Gender)
        mlp_embage = self.mlp_embage(Age)
        mlp_embocc = self.mlp_embocc(Occupation)
        mlp_movieid = self.mlp_embmovieid(MovieID)

        mlp_vector = torch.cat([mlp_embuserid,
                                mlp_embgender,
                                mlp_embage,
                                mlp_embocc,
                                mlp_movieid,
                                movie_ohe],
                                dim=1)
        mlp_vector = F.relu(self.dropout1(self.h1(mlp_vector)))
        mlp_vector = F.relu(self.dropout2(self.h2(mlp_vector)))
        #mlp_vector = F.relu(self.dropout3(self.h3(mlp_vector)))

        # Fusion
        result = torch.cat([gmf_vector, mlp_vector], dim=1)
        result = self.last_layer(result)

        #return (torch.sigmoid(result) * (5-1) + 1).squeeze
        return (torch.sigmoid(result) * (self.y_range[1]-self.y_range[0]) + self.y_range[0]).squeeze()


model = ncf(train.users_emb, train.movies_emb, train.users_ohe, train.movies_ohe, train.interact, 60, 0.5, 200, train.y_range)
model.to(device)
print(model)

ncf(
  (gmf_embuserid): Embedding(943, 60)
  (gmf_embgender): Embedding(2, 60)
  (gmf_embage): Embedding(7, 60)
  (gmf_embocc): Embedding(21, 60)
  (gmf_embmovieid): Embedding(1682, 221)
  (mlp_embuserid): Embedding(943, 60)
  (mlp_embgender): Embedding(2, 60)
  (mlp_embage): Embedding(7, 60)
  (mlp_embocc): Embedding(21, 60)
  (mlp_embmovieid): Embedding(1682, 60)
  (h1): Linear(in_features=319, out_features=200, bias=True)
  (h2): Linear(in_features=200, out_features=100, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (last_layer): Linear(in_features=340, out_features=1, bias=True)
)


### Skorch callbacks

In [ ]:
# Earlystopping callback
earlystopping = EarlyStopping(monitor='valid_loss', patience=10, threshold=0.001)

In [ ]:
# RMSE callback
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

def rmseloss(y_true, y_pred):
    #return f1_score(y_true, y_pred)
    return np.sqrt(mean_squared_error(y_true, y_pred))

def precision(y_true, y_pred):
    y_true_binary = (y_true >= 4).astype(int)
    y_pred_binary = (y_pred >= 4).astype(int)

    # Calculate Precision and Recall
    precision_scoree = precision_score(y_true_binary, y_pred_binary)
    return precision_scoree

def recall(y_true, y_pred):
    y_true_binary = (y_true >= 4).astype(int)
    y_pred_binary = (y_pred >= 4).astype(int)

    # Calculate Precision and Recall
    recall_scoree = recall_score(y_true_binary, y_pred_binary)
    return recall_scoree

def f1(y_true, y_pred):
    y_true_binary = (y_true >= 4).astype(int)
    y_pred_binary = (y_pred >= 4).astype(int)

    # Calculate F1 score
    f1_scoree = f1_score(y_true_binary, y_pred_binary)
    return f1_scoree

rmse_scorer = make_scorer(rmseloss)
precision_scorer = make_scorer(precision)
recall_scorer = make_scorer(recall)
f1_scorer = make_scorer(f1)

epoch_rmse = EpochScoring(rmse_scorer, name='rmse_score', lower_is_better=True)
epoch_precision = EpochScoring(precision_scorer, name='precision', lower_is_better= False)
epoch_recall = EpochScoring(recall_scorer, name='recall', lower_is_better= False)
epoch_f1 = EpochScoring(f1_scorer, name='f1', lower_is_better= False)


In [ ]:
# Checkpoint callback
checkpoint = Checkpoint(monitor='rmse_score_best', f_params='params.pt', f_optimizer='optimizer.pt', f_history='history.json', f_pickle='model')

In [ ]:
# Learning rate scheduler callback
lr_scheduler = LRScheduler(policy="StepLR", step_size=7, gamma=0.1)

### Neural Collaborative Filtering

#### Manually specify hyperparamers

In [ ]:
#86.57
ncfnet = NeuralNet(
    ncf,
    module__users_emb=train.users_emb,
    module__movies_emb=train.movies_emb,
    module__users_ohe=train.users_ohe,
    module__movies_ohe=train.movies_ohe,
    module__interact=train.interact,
    module__size_emb=120,
    module__dropout=0.3,
    module__linear_size=400,
    module__y_range=train.y_range,#### Manually specify hyperparamers
    max_epochs=50,
    lr=0.01,
    optimizer=torch.optim.Adam,
    criterion=torch.nn.MSELoss,
    device=device,
    iterator_train__batch_size=1024,
    iterator_train__num_workers=0,
    iterator_train__shuffle=True,
    iterator_valid__batch_size=4096,
    train_split=predefined_split(valid_dataset),
    callbacks=[
               earlystopping,
               epoch_rmse,
               epoch_precision,
               epoch_recall,
               epoch_f1,
               checkpoint,
               lr_scheduler,

               ]
)

In [ ]:
ncfnet.fit(train_dataset)

  epoch      f1    precision    recall    rmse_score    train_loss    valid_loss    cp      lr     dur
-------  ------  -----------  --------  ------------  ------------  ------------  ----  ------  ------
      1  0.4456       0.8579    0.3010        0.9324        1.0012        0.8695     +  0.0100  3.6711
      2  0.5758       0.7997    0.4498        0.9671        0.6406        0.9354        0.0100  2.9789
      3  0.5740       0.7998    0.4477        0.9821        0.2568        0.9646        0.0100  2.7147
      4  0.6083       0.7849    0.4966        0.9922        0.1416        0.9846        0.0100  2.7281
      5  0.5872       0.7791    0.4711        1.0081        0.1021        1.0162        0.0100  3.7918
      6  0.6089       0.7735    0.5021        1.0133        0.0865        1.0268        0.0100  2.7138
      7  0.6048       0.7771    0.4951        1.0129        0.0793        1.0261        0.0100  2.7232
      8  0.5978       0.7771    0.4858        1.0117        0.0670       

<class 'skorch.net.NeuralNet'>[initialized](
  module_=ncf(
    (gmf_embuserid): Embedding(943, 120)
    (gmf_embgender): Embedding(2, 120)
    (gmf_embage): Embedding(7, 120)
    (gmf_embocc): Embedding(21, 120)
    (gmf_embmovieid): Embedding(1682, 461)
    (mlp_embuserid): Embedding(943, 120)
    (mlp_embgender): Embedding(2, 120)
    (mlp_embage): Embedding(7, 120)
    (mlp_embocc): Embedding(21, 120)
    (mlp_embmovieid): Embedding(1682, 120)
    (h1): Linear(in_features=619, out_features=400, bias=True)
    (h2): Linear(in_features=400, out_features=200, bias=True)
    (dropout1): Dropout(p=0.3, inplace=False)
    (dropout2): Dropout(p=0.3, inplace=False)
    (last_layer): Linear(in_features=680, out_features=1, bias=True)
  ),
)

#### GridSearchCV

In [ ]:
params = {
    'lr': [0.01],
    'module__size_emb': [120],
    'module__dropout': [0.3],
    'module__linear_size': [400]
}
gs = GridSearchCV(ncfnet,
                  params,
                  verbose=50,
                  refit=True,
                  pre_dispatch=2,
                  n_jobs=1,
                  cv=5,
                  scoring='neg_mean_squared_error')

X_ds = SliceDataset(train, idx=0)
y_ds = SliceDataset(train, idx=1)
gs.fit(X_ds, y_ds)

print(gs.best_score_, gs.best_params_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START lr=0.01, module__dropout=0.3, module__linear_size=400, module__size_emb=120
  epoch      f1    precision    recall    rmse_score    train_loss    valid_loss    cp      lr     dur
-------  ------  -----------  --------  ------------  ------------  ------------  ----  ------  ------
      1  0.5496       0.8684    0.4020        0.8891        0.9943        0.7905     +  0.0100  5.8627
      2  0.7129       0.9620    0.5662        0.6510        0.6327        0.4238     +  0.0100  4.8760
      3  0.7997       0.9718    0.6794        0.5762        0.2678        0.3320     +  0.0100  5.4818
      4  0.8044       0.9770    0.6837        0.5479        0.1509        0.3002     +  0.0100  6.7898
      5  0.7913       0.9772    0.6649        0.5381        0.1090        0.2896     +  0.0100  6.4976
      6  0.7753       0.9791    0.6418        0.5338        0.0911        0.2849     +  0.0100  4.2912
      7  0.8243      

In [ ]:
best_model = gs.best_estimator_
best_model.fit(train_dataset)

Re-initializing module because the following parameters were re-set: dropout, interact, linear_size, movies_emb, movies_ohe, size_emb, users_emb, users_ohe, y_range.
Re-initializing criterion.
Re-initializing optimizer.
  epoch      f1    precision    recall    rmse_score    train_loss    valid_loss    cp      lr     dur
-------  ------  -----------  --------  ------------  ------------  ------------  ----  ------  ------
      1  0.4265       0.8679    0.2827        0.9295        0.9991        0.8639     +  0.0100  3.8039
      2  0.6672       0.7917    0.5765        0.9653        0.6396        0.9319        0.0100  3.0519
      3  0.6050       0.7948    0.4884        0.9748        0.2639        0.9502        0.0100  2.9263
      4  0.6203       0.7937    0.5091        0.9864        0.1437        0.9730        0.0100  2.7757
      5  0.6333       0.7850    0.5308        0.9986        0.1035        0.9971        0.0100  3.8970
      6  0.6020       0.7855    0.4880        1.0102       

<class 'skorch.net.NeuralNet'>[initialized](
  module_=ncf(
    (gmf_embuserid): Embedding(943, 120)
    (gmf_embgender): Embedding(2, 120)
    (gmf_embage): Embedding(7, 120)
    (gmf_embocc): Embedding(21, 120)
    (gmf_embmovieid): Embedding(1682, 461)
    (mlp_embuserid): Embedding(943, 120)
    (mlp_embgender): Embedding(2, 120)
    (mlp_embage): Embedding(7, 120)
    (mlp_embocc): Embedding(21, 120)
    (mlp_embmovieid): Embedding(1682, 120)
    (h1): Linear(in_features=619, out_features=400, bias=True)
    (h2): Linear(in_features=400, out_features=200, bias=True)
    (dropout1): Dropout(p=0.3, inplace=False)
    (dropout2): Dropout(p=0.3, inplace=False)
    (last_layer): Linear(in_features=680, out_features=1, bias=True)
  ),
)

In [ ]:
#rating_predict = best_model.predict(valid_dataset)
rating_predict = ncfnet.predict(valid_dataset)

In [ ]:
valid_users = valid_dataset[:][0][0][:,0].numpy()
valid_movie = valid_dataset[:][0][0][:,3].numpy()
valid_rating = valid_dataset[:][1].numpy()

In [ ]:
dct= {}
for i in range(20000):
    if valid_users[i] not in dct.keys():
        dct[valid_users[i]]  = [(i, valid_movie[i], rating_predict[i], valid_rating[i])]
    else:
        dct[valid_users[i]].append((i, valid_movie[i], rating_predict[i], valid_rating[i]))

In [ ]:
#calculate HR@10
from sklearn.metrics import ndcg_score
import math
cnt_user = 0
sum = 0
ndcg = 0
for key, value in dct.items():
    if len(value) >= 10:
        cnt_user += 1
        cnt_movie = 0
        value_rank_by_pred = sorted(value, reverse = True , key=lambda x: x[2]) # sort by pred
        value_rank_by_label = sorted(value, reverse = True , key=lambda x: x[3]) # sort by label
        ranked_label = [[item[3] for item in value_rank_by_label]]
        ranked_pred = [[item[3] for item in value_rank_by_pred]]
        dcg = 0
        idcg = 0
        for j in range(10):
            if value_rank_by_pred[j][3] == 5:
                sum += 1
                break
        for j in range(10):
            dcg += (2**ranked_pred[0][j]-1)/(math.log2(j+2))
            idcg += (2**ranked_label[0][j]-1)/(math.log2(j+2))
        ndcg += dcg/idcg
        #ndcg += ndcg_score(ranked_label, ranked_pred, k=10)
        #ndcg += ndcg_at_k(ranked_pred, ranked_label, 10)
hr = sum/cnt_user
ndcg = ndcg/cnt_user
print(hr, ndcg)

0.9092495636998255 0.7560052037481741


In [ ]:
torch.save(best_model.module_.state_dict(), 'best_model.pth')

### Deep and Wide

#### Manually specify hyperparamers

In [ ]:
deepnwidenet = NeuralNet(
    deepnwide,
    module__users_emb=train.users_emb,
    module__movies_emb=train.movies_emb,
    module__users_ohe=train.users_ohe,
    module__movies_ohe=train.movies_ohe,
    #### Manually specify hyperparamers e=train.movies_ohe,
    module__interact=train.interact,
    module__size_emb=120,
    module__y_range=train.y_range,
    module__dropout=0.2,
    max_epochs=30,
    lr=0.01,
    optimizer=torch.optim.Adam,
    criterion=torch.nn.MSELoss,
    device=device,
    iterator_train__batch_size=1024,
    iterator_train__num_workers=0,
    iterator_train__shuffle=True,
    iterator_valid__batch_size=4096,
    train_split=predefined_split(valid_dataset),
    callbacks=[
               earlystopping,
               epoch_rmse,
               epoch_precision,
               epoch_recall,
               epoch_f1,
               #checkpoint,
               lr_scheduler,
               #TensorBoard(writer),
               #progressbar
               ]
)

In [ ]:
deepnwidenet.fit(train_dataset)

  epoch      f1    precision    recall    rmse_score    train_loss    valid_loss      lr     dur
-------  ------  -----------  --------  ------------  ------------  ------------  ------  ------
      1  0.4080       0.8612    0.2673        0.9489        0.9904        0.9005  0.0100  2.9244
      2  0.3391       0.8828    0.2099        0.9444        0.8754        0.8919  0.0100  2.7907
      3  0.5046       0.8392    0.3608        0.9349        0.8563        0.8741  0.0100  3.0387
      4  0.5105       0.8395    0.3668        0.9383        0.8406        0.8804  0.0100  3.3135
      5  0.4705       0.8484    0.3255        0.9342        0.8244        0.8728  0.0100  3.2297
      6  0.5033       0.8464    0.3582        0.9335        0.8125        0.8713  0.0100  2.9621
      7  0.5712       0.8295    0.4356        0.9467        0.7965        0.8962  0.0100  3.4203
      8  0.5513       0.8404    0.4102        0.9275        0.7385        0.8602  0.0010  3.8236
      9  0.5654       0.8372  

<class 'skorch.net.NeuralNet'>[initialized](
  module_=deepnwide(
    (emb_UserID): Embedding(943, 120)
    (emb_Gender): Embedding(2, 120)
    (emb_Age): Embedding(7, 120)
    (emb_Occupation): Embedding(21, 120)
    (emb_MovieID): Embedding(1682, 120)
    (h1): Linear(in_features=600, out_features=500, bias=True)
    (h2): Linear(in_features=500, out_features=500, bias=True)
    (h3): Linear(in_features=500, out_features=500, bias=True)
    (dropout1): Dropout(p=0.2, inplace=False)
    (dropout2): Dropout(p=0.2, inplace=False)
    (dropout3): Dropout(p=0.2, inplace=False)
    (last_layer): Linear(in_features=845, out_features=1, bias=True)
  ),
)

#### GridsearchCV

In [ ]:
# params = {
#     'lr': [0.001, 0.01],
#     'module__size_emb': [30, 60, 120],
#     'module__dropout': [0.5],
#     'module__linear_size': [400, 500, 600]
# }
params = {
    'lr': [0.01],
    'module__size_emb': [60, 120],
    'module__dropout': [0.3],
    'module__linear_size': [500, 1000, 1500]
}
gs = GridSearchCV(deepnwidenet,
                  params,
                  verbose=50,
                  refit=True,
                  #pre_dispatch=8,
                  n_jobs=1,
                  cv=3,
                  scoring='neg_mean_squared_error')

X_ds = SliceDataset(train, idx=0)
y_ds = SliceDataset(train, idx=1)
gs.fit(X_ds, y_ds)

print(gs.best_score_, gs.best_params_)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV 1/3; 1/6] START lr=0.01, module__dropout=0.3, module__linear_size=500, module__size_emb=60
  epoch      f1    precision    recall    rmse_score    train_loss    valid_loss      lr     dur
-------  ------  -----------  --------  ------------  ------------  ------------  ------  ------
      1  0.3490       0.8462    0.2198        0.9818        1.0217        0.9640  0.0100  5.5711
      2  0.5170       0.8149    0.3786        0.9667        0.8876        0.9345  0.0100  4.9588
      3  0.3983       0.8361    0.2614        0.9621        0.8652        0.9257  0.0100  4.4296
      4  0.3291       0.8498    0.2041        0.9646        0.8502        0.9305  0.0100  3.9035
      5  0.3477       0.8433    0.2190        0.9743        0.8441        0.9492  0.0100  3.7193
      6  0.4528       0.8334    0.3108        0.9453        0.8312        0.8935  0.0100  3.9781
      7  0.3009       0.8653    0.1821        0.9542        0.8180   

In [ ]:
best_model = gs.best_estimator_
best_model.fit(train_dataset)

Re-initializing module because the following parameters were re-set: dropout, interact, linear_size, movies_emb, movies_ohe, size_emb, users_emb, users_ohe, y_range.
Re-initializing criterion.
Re-initializing optimizer.
  epoch      f1    precision    recall    rmse_score    train_loss    valid_loss      lr     dur
-------  ------  -----------  --------  ------------  ------------  ------------  ------  ------
      1  0.5195       0.8302    0.3780        0.9388        1.0714        0.8814  0.0100  4.1835
      2  0.6028       0.8075    0.4809        0.9449        0.8874        0.8929  0.0100  4.2092
      3  0.5556       0.8149    0.4215        0.9366        0.8700        0.8773  0.0100  3.6634
      4  0.4168       0.8626    0.2748        0.9334        0.8507        0.8712  0.0100  3.8046
      5  0.5149       0.8460    0.3701        0.9274        0.8373        0.8601  0.0100  4.1308
      6  0.5223       0.8401    0.3789        0.9294        0.8246        0.8638  0.0100  3.8217
    

<class 'skorch.net.NeuralNet'>[initialized](
  module_=deepnwide(
    (emb_UserID): Embedding(943, 120)
    (emb_Gender): Embedding(2, 120)
    (emb_Age): Embedding(7, 120)
    (emb_Occupation): Embedding(21, 120)
    (emb_MovieID): Embedding(1682, 120)
    (h1): Linear(in_features=600, out_features=1000, bias=True)
    (h2): Linear(in_features=1000, out_features=1000, bias=True)
    (h3): Linear(in_features=1000, out_features=1000, bias=True)
    (dropout1): Dropout(p=0.3, inplace=False)
    (dropout2): Dropout(p=0.3, inplace=False)
    (dropout3): Dropout(p=0.3, inplace=False)
    (last_layer): Linear(in_features=1345, out_features=1, bias=True)
  ),
)

In [ ]:
#rating_predict = best_model.predict(valid_dataset)
rating_predict = deepnwidenet.predict(valid_dataset)

In [40]:
valid_users = valid_dataset[:][0][0][:,0].numpy()
valid_movie = valid_dataset[:][0][0][:,3].numpy()
valid_rating = valid_dataset[:][1].numpy()

In [ ]:
valid_rating

In [41]:
dct= {}
for i in range(20000):
    if valid_users[i] not in dct.keys():
        dct[valid_users[i]]  = [(i, valid_movie[i], rating_predict[i], valid_rating[i])]
    else:
        dct[valid_users[i]].append((i, valid_movie[i], rating_predict[i], valid_rating[i]))

In [ ]:
dct[725][1]

In [42]:
#calculate HR@10
from sklearn.metrics import ndcg_score
import math
cnt_user = 0
sum = 0
ndcg = 0
for key, value in dct.items():
    if len(value) >= 10:
        cnt_user += 1
        cnt_movie = 0
        value_rank_by_pred = sorted(value, reverse = True , key=lambda x: x[2]) # sort by pred
        value_rank_by_label = sorted(value, reverse = True , key=lambda x: x[3]) # sort by label
        ranked_label = [[item[3] for item in value_rank_by_label]]
        ranked_pred = [[item[3] for item in value_rank_by_pred]]
        dcg = 0
        idcg = 0
        for j in range(10):
            if value_rank_by_pred[j][3] == 5:
                sum += 1
                break
        for j in range(10):
            dcg += (2**ranked_pred[0][j]-1)/(math.log2(j+2))
            idcg += (2**ranked_label[0][j]-1)/(math.log2(j+2))
        ndcg += dcg/idcg
        #ndcg += ndcg_score(ranked_label, ranked_pred, k=10)
        #ndcg += ndcg_at_k(ranked_pred, ranked_label, 10)
hr = sum/cnt_user
ndcg = ndcg/cnt_user
print(hr, ndcg)

0.9144851657940664 0.7691176541141781


### Two embeddings - Basic matrix factorization

#### Manually specify hyperparamers

In [43]:
twoembedsnet = NeuralNet(
    twoembeds,
    module__size_emb=500,
    module__y_range=train.y_range,
    max_epochs=30,
    lr=0.01,
    optimizer=torch.optim.Adam,
    criterion=torch.nn.MSELoss,
    device=device,
    iterator_train__batch_size=4096,
    iterator_train__num_workers=0,
    iterator_train__shuffle=True,
    iterator_valid__batch_size=4096,
    train_split=predefined_split(valid_dataset),
    callbacks=[earlystopping,
               epoch_rmse,
               epoch_precision,
               epoch_recall,
               epoch_f1,
               #checkpoint,
               lr_scheduler]
)

In [44]:
twoembedsnet.fit(train_dataset)

  epoch      f1    precision    recall    rmse_score    train_loss    valid_loss      lr     dur
-------  ------  -----------  --------  ------------  ------------  ------------  ------  ------
      1  0.6073       0.7948    0.4914        0.9753        1.2419        0.9512  0.0100  4.1760
      2  0.4658       0.8719    0.3178        0.9309        0.6232        0.8665  0.0100  3.1107
      3  0.5314       0.8394    0.3887        0.9445        0.2534        0.8921  0.0100  3.1655
      4  0.5495       0.8286    0.4111        0.9556        0.1141        0.9131  0.0100  3.1317
      5  0.5524       0.8302    0.4139        0.9595        0.0713        0.9206  0.0100  3.9620
      6  0.5625       0.8242    0.4269        0.9623        0.0561        0.9261  0.0100  3.2080
      7  0.5644       0.8228    0.4295        0.9674        0.0527        0.9358  0.0100  2.8956
      8  0.5650       0.8234    0.4300        0.9658        0.0431        0.9328  0.0010  3.3355
      9  0.5652       0.8239  

<class 'skorch.net.NeuralNet'>[initialized](
  module_=twoembeds(
    (emb_UserID): Embedding(943, 500)
    (emb_MovieID): Embedding(1682, 500)
    (emb_UserID_b): Embedding(943, 1)
    (emb_MovieID_b): Embedding(1682, 1)
  ),
)

In [ ]:
torch.save(twoembedsnet.module_.state_dict(), 'GMF.pth')

#### GridSearchCV

In [ ]:
params = {
    'lr': [0.01],
    'module__size_emb': [500]
}
gs = GridSearchCV(twoembedsnet,
                  params,
                  verbose=50,
                  refit=True,
                  #pre_dispatch=8,
                  #n_jobs=8,
                  cv=3,
                  scoring='neg_mean_squared_error')

X_ds = SliceDataset(train, idx=0)
y_ds = SliceDataset(train, idx=1)
gs.fit(X_ds, y_ds)
print(gs.best_score_, gs.best_params_)





Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3; 1/1] START lr=0.01, module__size_emb=500...............................
  epoch      f1    precision    recall    rmse_score    train_loss    valid_loss      lr     dur
-------  ------  -----------  --------  ------------  ------------  ------------  ------  ------
      1  0.5349       0.8186    0.3972        0.9922        1.2497        0.9846  0.0100  3.9989
      2  0.4981       0.9545    0.3370        0.8469        0.6804        0.7173  0.0100  4.1132
      3  0.6667       0.9865    0.5035        0.7497        0.3272        0.5621  0.0100  4.7773
      4  0.7077       0.9964    0.5488        0.7136        0.1508        0.5092  0.0100  4.2700
      5  0.7063       0.9978    0.5466        0.6989        0.0878        0.4885  0.0100  4.2813
      6  0.7001       0.9988    0.5389        0.6915        0.0615        0.4782  0.0100  4.5627
      7  0.6831       0.9990    0.5190        0.6885        0.0493        0.4740  0.

In [ ]:
best_model = gs.best_estimator_
best_model.fit(train_dataset)

Re-initializing module because the following parameters were re-set: size_emb, y_range.
Re-initializing criterion.
Re-initializing optimizer.
  epoch      f1    precision    recall    rmse_score    train_loss    valid_loss      lr     dur
-------  ------  -----------  --------  ------------  ------------  ------------  ------  ------
      1  0.6109       0.7961    0.4956        0.9753        1.2464        0.9513  0.0100  3.8133
      2  0.4767       0.8682    0.3285        0.9236        0.6205        0.8530  0.0100  3.5169
      3  0.5336       0.8474    0.3894        0.9374        0.2516        0.8787  0.0100  3.5038
      4  0.5525       0.8289    0.4144        0.9485        0.1132        0.8996  0.0100  4.3620
      5  0.5533       0.8289    0.4153        0.9534        0.0728        0.9090  0.0100  3.5139
      6  0.5630       0.8238    0.4276        0.9551        0.0576        0.9122  0.0100  3.5021
      7  0.5660       0.8182    0.4326        0.9606        0.0539        0.9227  

<class 'skorch.net.NeuralNet'>[initialized](
  module_=twoembeds(
    (emb_UserID): Embedding(943, 500)
    (emb_MovieID): Embedding(1682, 500)
    (emb_UserID_b): Embedding(943, 1)
    (emb_MovieID_b): Embedding(1682, 1)
  ),
)

In [45]:
#rating_predict = best_model.predict(valid_dataset)
rating_predict = twoembedsnet.predict(valid_dataset)

In [46]:
valid_users = valid_dataset[:][0][0][:,0].numpy()
valid_movie = valid_dataset[:][0][0][:,3].numpy()
valid_rating = valid_dataset[:][1].numpy()

In [47]:
dct= {}
for i in range(20000):
    if valid_users[i] not in dct.keys():
        dct[valid_users[i]]  = [(i, valid_movie[i], rating_predict[i], valid_rating[i])]
    else:
        dct[valid_users[i]].append((i, valid_movie[i], rating_predict[i], valid_rating[i]))

In [48]:
#calculate HR@10
from sklearn.metrics import ndcg_score
import math
cnt_user = 0
sum = 0
ndcg = 0
for key, value in dct.items():
    if len(value) >= 10:
        cnt_user += 1
        cnt_movie = 0
        value_rank_by_pred = sorted(value, reverse = True , key=lambda x: x[2]) # sort by pred
        value_rank_by_label = sorted(value, reverse = True , key=lambda x: x[3]) # sort by label
        ranked_label = [[item[3] for item in value_rank_by_label]]
        ranked_pred = [[item[3] for item in value_rank_by_pred]]
        dcg = 0
        idcg = 0
        for j in range(10):
            if value_rank_by_pred[j][3] == 5:
                sum += 1
                break
        for j in range(10):
            dcg += (2**ranked_pred[0][j]-1)/(math.log2(j+2))
            idcg += (2**ranked_label[0][j]-1)/(math.log2(j+2))
        ndcg += dcg/idcg
        #ndcg += ndcg_score(ranked_label, ranked_pred, k=10)
        #ndcg += ndcg_at_k(ranked_pred, ranked_label, 10)
hr = sum/cnt_user
ndcg = ndcg/cnt_user
print(hr, ndcg, cnt_user)

0.9057591623036649 0.7721473278004224 573


### Benchmark with scikit-surprise SVD algorithm

In [ ]:
!pip install surprise

In [ ]:
from surprise import NormalPredictor
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import cross_validate, train_test_split, KFold

In [ ]:
train[dataloaders['train'].dataset.indices][1]

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

# Load the movielens-100k dataset (download it if needed).
data = Dataset.load_builtin('ml-100k')

# Use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

#Predict

In [ ]:
movies = pd.read_csv('/content/ml-100k/u.item', sep='|', names=['MovieID', 'Title', 'date', 'video_rl_date', 'link']+genre_cols, engine='python', encoding='latin-1')
users = pd.read_csv('/content/ml-100k/u.user', sep='|', names=['UserID', 'Age', 'Gender', 'Occupation', 'Zipcode'], engine='python', encoding='latin-1')
ratings = pd.read_csv('/content/ml-100k/u.data', sep='\t', names=['UserID', 'MovieID', 'Rating', 'Timestamp'], engine='python', encoding='latin-1')


In [ ]:
ratings

In [ ]:
filtered_rating = ratings[(ratings['UserID'] == 0) & (ratings['MovieID'] == 168)]['Rating'].values[0]
filtered_rating

In [ ]:
bins = [0, 18, 25, 35, 45, 50, 56, 100]
labels = [1, 18, 25, 35, 45, 50, 56]
users['Age'] = pd.cut(users['Age'], bins=bins, labels=labels, right=False)

In [ ]:
movies

In [ ]:
# Label Encode users
columns = ['UserID', 'Gender', 'Age', 'Occupation']
users[columns] = users[columns].apply(preprocessing.LabelEncoder().fit_transform)
#users_emb_columns = train.users_emb_columns + columns
users = users.drop(['Zipcode'], axis = 1)

In [ ]:
# One Hot Encode users
users_ohe_columns = []
columns = ['Gender', 'Age', 'Occupation']
ohe = preprocessing.OneHotEncoder(categories='auto', sparse=False, dtype='uint8')
ohe.fit(train.ratings[columns])
users = pd.concat([users, pd.DataFrame(data=ohe.transform(train.ratings[columns]), columns=ohe.get_feature_names_out(columns))], axis=1)
users_ohe_columns = ohe.get_feature_names_out(columns)


In [ ]:
# Label Encode movies
columns = ['MovieID']
movies[columns] = movies[columns].apply(preprocessing.LabelEncoder().fit_transform)

In [ ]:
df2 = movies[genre_cols]
df2['Genre'] = df2.apply(lambda row: ', '.join(row.index[row == 1]), axis=1)
movies['Genre'] = df2['Genre']
movies = movies.drop(genre_cols, axis = 1)

In [ ]:
# One Hot Encode movies
movies_ohe_columns = []
genres = ["genre_unknown", "Action", "Adventure", "Animation", "Children", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "FilmNoir", "Horror",
    "Musical", "Mystery", "Romance", "SciFi", "Thriller", "War", "Western"]

for genre in genres:
    genre = genre.replace('-', '')
    column = str(genre)
    movies[column] = movies['Genre'].apply(lambda x: 1 if genre in x else 0)
    movies_ohe_columns.append(column)



In [ ]:
users_emb = torch.from_numpy(users[['UserID', 'Gender', 'Age', 'Occupation']].values)

In [ ]:
movies_emb = torch.from_numpy(movies['MovieID'].values)


In [ ]:
movies_emb[0].unsqueeze(0).unsqueeze(0)[:, 0]

In [ ]:
movies_ohe = torch.tensor(movies[movies_ohe_columns].values, dtype=torch.float)

In [ ]:
movies_ohe[0]

In [ ]:
users_emb[2].long()

In [ ]:
users_ohe = torch.tensor(users[users_ohe_columns].values, dtype=torch.float)


In [ ]:
def predict(i, model):
    lst_movie = []
    for j in range(len(movies)):
        X = (users_emb[i].long().unsqueeze(0), users_ohe[i].unsqueeze(0), movies_emb[j].long().unsqueeze(0).unsqueeze(0), movies_ohe[j].unsqueeze(0))

        y = model(X)
        if y > 4:
            lst_movie.append((movies_emb[j].long(), y.item()))
        lst_movie = sorted(lst_movie, key=lambda x: x[1], reverse = True)
    return lst_movie[:10]

In [ ]:

model = ncf(train.users_emb, train.movies_emb, train.users_ohe, train.movies_ohe, train.interact, 120, 0.5, 400, train.y_range)
model.to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/best_model.pth'))

In [ ]:
dictt = torch.load('/content/params.pt')


In [ ]:
#input is a user, return top 5 movies
predict(0, model)

In [ ]:
train_dataset[507][1]

In [ ]:
cnt = 0
for i in range(943):
    predict_ = predict(i, model)
    print(predict_)
    for j in range(len(predict_)):
        if (ratings[(ratings['UserID'] == i) & (ratings['MovieID'] == predict_[j][0].item())]['Rating'].values[0] == 5):
            cnt+= 1
cnt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
print(cnt/9430)